In [31]:
import tensorflow as tf
import tensorflow_data_validation as tfdv

from tfx import v1 as tfx

from tfx.types import standard_artifacts

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2

import os
import pprint
pp = pprint.PrettyPrinter()

In [32]:
_pipeline_root = './pipeline/'
_data_filepath = '/content/data1/adult.data'

In [33]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [34]:
import pandas as pd
from tfx.components import CsvExampleGen

In [35]:
df = pd.read_csv(_data_filepath, delimiter=',', skipinitialspace=True)

print(df.head())

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
                'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
                'hours_per_week', 'native_country', 'income']
df.columns = column_names


df.to_csv(_data_filepath, index=False)

   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country  label  
0          2174             0              40  United-States  <=50K  
1             0             0             

In [36]:
example_gen = tfx.components.CsvExampleGen(input_base='/content/data1/')

In [37]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [38]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [39]:
statistics_list = statistics_gen.outputs['statistics'].get()
if statistics_list:
    print(statistics_list[0].uri)
else:
    print("No statistics were generated.")
    # Invest

./pipeline/StatisticsGen/statistics/8


In [40]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [41]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital_gain',INT,required,,-
'capital_loss',INT,required,,-
'education',STRING,required,,'education'
'education_num',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-
'income',STRING,required,,'income'
'marital_status',STRING,required,,'marital_status'


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'income',"'<=50K', '>50K'"
'marital_status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native_country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'sex',"'Female', 'Male'"
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


In [42]:
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [43]:
tfdv.set_domain(schema, 'age', schema_pb2.IntDomain(name='age', min=17, max=90))

tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,min: 17; max: 90
'capital_gain',INT,required,,-
'capital_loss',INT,required,,-
'education',STRING,required,,'education'
'education_num',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-
'income',STRING,required,,'income'
'marital_status',STRING,required,,'marital_status'


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'income',"'<=50K', '>50K'"
'marital_status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native_country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'sex',"'Female', 'Male'"
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


In [44]:
print('label' in [feature.name for feature in schema.feature])


False


In [45]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

tfdv.get_feature(schema, 'label').not_in_environment.append('SERVING')

In [46]:
_updated_schema_dir = f'{_pipeline_root}/updated_schema'

!mkdir -p {_updated_schema_dir}

schema_file = os.path.join(_updated_schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

In [47]:
user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)

context.run(user_schema_importer, enable_cache=False)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 10
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [48]:
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,min: 17; max: 90
'capital_gain',INT,required,,-
'capital_loss',INT,required,,-
'education',STRING,required,,'education'
'education_num',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-
'income',STRING,required,,'income'
'marital_status',STRING,required,,'marital_status'


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'income',"'<=50K', '>50K'"
'marital_status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native_country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'sex',"'Female', 'Male'"
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


In [49]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=user_schema_importer.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 11
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [50]:
context.show(example_validator.outputs['anomalies'])

In [51]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

connection_config = context.metadata_connection_config

store = mlmd.MetadataStore(connection_config)

In [52]:
artifact_types = store.get_artifact_types()

[artifact_type.name for artifact_type in artifact_types]

['Examples', 'Schema', 'ExampleStatistics', 'ExampleAnomalies']

In [53]:
schema_list = store.get_artifacts_by_type('Schema')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/SchemaGen/schema/3', 'schema id:3'),
 ('schema uri: ./pipeline/SchemaGen/schema/4', 'schema id:4'),
 ('schema uri: ./pipeline/SchemaGen/schema/6', 'schema id:6'),
 ('schema uri: ./pipeline/SchemaGen/schema/9', 'schema id:9'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/10', 'schema id:10')]

In [55]:
example_anomalies = store.get_artifacts_by_type('ExampleAnomalies')[0]

print(f'Artifact id: {example_anomalies.id}')

Artifact id: 11


In [56]:
anomalies_id_event = store.get_events_by_artifact_ids([example_anomalies.id])[0]

print(anomalies_id_event)

artifact_id: 11
execution_id: 11
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1720650990001



In [57]:
anomalies_execution_id = anomalies_id_event.execution_id

events_execution = store.get_events_by_execution_ids([anomalies_execution_id])

print(events_execution)

[artifact_id: 8
execution_id: 11
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1720650989972
, artifact_id: 10
execution_id: 11
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1720650989973
, artifact_id: 11
execution_id: 11
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1720650990001
]


In [58]:
inputs_to_exval = [event.artifact_id for event in events_execution
                       if event.type == metadata_store_pb2.Event.INPUT]

print(inputs_to_exval)

[8, 10]
